In [1]:
import os
import pandas as pd
import numpy as np

In [186]:
train_dataset = pd.read_csv("Training.csv")

EmptyDataError: No columns to parse from file

In [160]:
test_dataset = pd.read_csv("Testing.csv")

In [94]:
train_dataset = train_dataset.dropna( axis=1, how='any',)
train_dataset.info


<bound method DataFrame.info of            ID product_type           sub_area  raion_popul  green_zone_part  \
0           1   Investment           Bibirevo       155572         0.189727   
1           2   Investment     Tekstil'shhiki       101708         0.112560   
2           3   Investment         Sokol'niki        57405         0.523439   
3           4   Investment            Koptevo        96959         0.038456   
4           5   Investment  Zapadnoe Degunino        78810         0.051844   
...       ...          ...                ...          ...              ...   
99995   99996   Investment    Molzhaninovskoe         3521         0.004676   
99996   99997   Investment    Molzhaninovskoe         3521         0.004676   
99997   99998   Investment    Molzhaninovskoe         3521         0.004676   
99998   99999   Investment    Molzhaninovskoe         3521         0.004676   
99999  100000   Investment    Molzhaninovskoe         3521         0.004676   

       indust_part 

In [161]:
targetColumn = train_dataset['price_doc']

In [162]:
train_dataset = train_dataset.drop(columns=['price_doc'])

In [163]:
train_dataset = train_dataset.drop(columns = ['ID'])
train_dataset = train_dataset.drop(columns = ['build_count_foam'])
train_dataset = train_dataset.drop(columns = ['mosque_count_2000'])
train_dataset = train_dataset.drop(columns = ['mosque_count_1000'])
train_dataset = train_dataset.drop(columns = ['school_education_centers_top_20_raion'])







In [164]:
test_dataset = test_dataset.drop(columns = ['ID'])
test_dataset = test_dataset.drop(columns = ['build_count_foam'])
test_dataset = test_dataset.drop(columns = ['mosque_count_2000'])
test_dataset = test_dataset.drop(columns = ['mosque_count_1000'])
test_dataset = test_dataset.drop(columns = ['school_education_centers_top_20_raion'])


In [183]:
train_dataset = train_dataset[['floor','market_count_5000']].round(0)
test_dataset = test_dataset.round(0)

KeyError: "None of [Index(['floor', 'market_count_5000'], dtype='object')] are in the [index]"

In [167]:
_NUMERIC_COLS_WITH_MISSING_VALUES = train_dataset.select_dtypes(include=[int,float])


In [168]:
# PREPROCESSING-1
from sklearn.impute import SimpleImputer
import numpy as np
for _col in _NUMERIC_COLS_WITH_MISSING_VALUES:
    __simple_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    train_dataset[_col] = __simple_imputer.fit_transform(train_dataset[_col].values.reshape(-1,1))[:,0]
    if _col in test_dataset:
        test_dataset[_col] = __simple_imputer.transform(test_dataset[_col].astype(train_dataset[_col].dtypes).values.reshape(-1,1))[:,0]

In [169]:
_STRING_COLS_WITH_MISSING_VALUES = train_dataset.select_dtypes(include=[object])

In [170]:
# PREPROCESSING-2
from sklearn.impute import SimpleImputer
import numpy as np
for _col in _STRING_COLS_WITH_MISSING_VALUES:
    __simple_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    train_dataset[_col] = __simple_imputer.fit_transform(train_dataset[_col].values.reshape(-1,1))[:,0]
    if _col in test_dataset:
        test_dataset[_col] = __simple_imputer.transform(test_dataset[_col].astype(train_dataset[_col].dtypes).values.reshape(-1,1))[:,0]

In [171]:
# PREPROCESSING-3
from sklearn.preprocessing import OrdinalEncoder
_CATEGORICAL_COLS = ['sub_area','product_type', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']
_ohe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
train_dataset[_CATEGORICAL_COLS] = pd.DataFrame(_ohe.fit_transform(train_dataset[_CATEGORICAL_COLS]), columns=_CATEGORICAL_COLS)
test_dataset[_CATEGORICAL_COLS] = pd.DataFrame(_ohe.transform(test_dataset[_CATEGORICAL_COLS]), columns=_CATEGORICAL_COLS)

In [172]:
train_dataset.info


<bound method DataFrame.info of        floor  product_type  sub_area  raion_popul  green_zone_part  \
0        4.0           0.0       6.0     155572.0              0.0   
1        2.0           0.0      96.0     101708.0              0.0   
2       10.0           0.0      89.0      57405.0              1.0   
3        5.0           0.0      33.0      96959.0              0.0   
4       11.0           0.0     106.0      78810.0              0.0   
...      ...           ...       ...          ...              ...   
99995    7.0           0.0      46.0       3521.0              0.0   
99996   10.0           0.0      46.0       3521.0              0.0   
99997    7.0           0.0      46.0       3521.0              0.0   
99998    8.0           0.0      46.0       3521.0              0.0   
99999    7.0           0.0      46.0       3521.0              0.0   

       indust_part  preschool_quota  preschool_education_centers_raion  \
0              0.0           5001.0                  

In [173]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor

In [ ]:

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
lreg = LinearRegression()
sfs1 = sfs(lreg, k_features=100, forward=True, verbose=2, scoring='neg_mean_squared_error')
sfs1 = sfs1.fit(train_dataset, targetColumn)
feat_names = list(sfs1.k_feature_names_)
ForwardSelectionTraining = train_dataset[feat_names]
ForwardSelectionTesting = test_dataset[feat_names]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 105 out of 105 | elapsed:    3.1s finished

[2022-11-18 17:33:21] Features: 1/100 -- score: -12861708095114.057[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 104 out of 104 | elapsed:    7.5s finished

[2022-11-18 17:33:29] Features: 2/100 -- score: -12147026328992.4[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 103 out of 103 | elapsed:   13.0s finished

[2022-11-18 17:33:42] Features: 3/100 -- score: -11917649687895.947[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Do

In [177]:
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['build_count_foam'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['mosque_count_1000'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['school_education_centers_top_20_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['incineration_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['railroad_terminal_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['oil_chemistry_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['culture_objects_top_25'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['big_market_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['university_top_20_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['thermal_power_plant_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['detention_facility_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['radiation_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['build_count_before_1920'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['preschool_education_centers_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['healthcare_centers_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['raion_popul'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['build_count_1946-1970'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['preschool_quota'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['additional_education_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['school_quota'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['build_count_1971-1995'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['raion_build_count_with_material_info'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['sport_objects_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['shopping_centers_raion'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['green_zone_part'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['build_count_monolith'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['build_count_brick'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['build_count_block'])
# ForwardSelectionTraining = ForwardSelectionTraining.drop(columns=['indust_part'])

In [178]:
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['build_count_foam'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['mosque_count_1000'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['school_education_centers_top_20_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['incineration_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['railroad_terminal_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['oil_chemistry_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['culture_objects_top_25'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['big_market_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['university_top_20_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['thermal_power_plant_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['detention_facility_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['radiation_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['build_count_before_1920'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['preschool_education_centers_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['healthcare_centers_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['raion_popul'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['build_count_1946-1970'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['preschool_quota'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['additional_education_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['school_quota'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['build_count_1971-1995'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['raion_build_count_with_material_info'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['sport_objects_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['shopping_centers_raion'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['green_zone_part'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['build_count_monolith'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['build_count_brick'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['build_count_block'])
# ForwardSelectionTesting = ForwardSelectionTesting.drop(columns=['indust_part'])

In [174]:
from sklearn.datasets import make_regression
from sklearn.ensemble import ExtraTreesRegressor
reg = ExtraTreesRegressor(random_state=0,n_estimators=200,verbose=2).fit( train_dataset, targetColumn)
targetColumn = reg.predict(test_dataset)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 200


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.9s remaining:    0.0s


building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 15.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    4.4s finished


In [175]:
import pandas
feature = pandas.DataFrame({'col_name': reg.feature_importances_}, index=train_dataset.columns).sort_values(by='col_name', ascending=False)

In [176]:
import csv
output = pd.read_csv('sample.csv')
output['price_doc'] = targetColumn
output.to_csv('etp3.csv', index = False)